# Processing MITGCM data for Scale Analysis 

**Purpose**: Code for producing data for scale analysis. I will be producing three types of data output for temperature and salinity: 

1. Point measurements on, off, and in the transition region of the shelf. 
2. Zonal and meridional spatial transects on, off, and in the transition region of the shelf. 

For statistical stability, I should average the autocorrelation functions over multiple points or multiple spatial transects. 

**Luke Colosi | lcolosi@ucsd.edu**

Force matplotlib plots to display directly within the output cell of the notebook: 

In [1]:
%matplotlib inline

Import python libraries

In [2]:
import sys
import xarray as xr
from xmitgcm import open_mdsdataset

#--- Other Functions ---# 
sys.path.append("/home/lcolosi/AirSeaCoupling/tools/")
import cartopy_figs as cart

Set data analysis parameters

In [3]:
# Model parameters 
delta_t = 150  # Time steps of the raw model run (by raw, I mean the time increments that the model is ran at, not time increments that the diagnostics are output at). Units: seconds

# Set time and space parameters  
lat_bnds  = [32.5, 35]                                           # Specifies the latitude bounds for the region to analyze
lon_bnds  = [236.0, 241.0]                                       # Specifies the longitude bounds for the region to analyze
encoding  = {'time': {'units': 'seconds since 2015-12-01 2:00'}} # Specifies the start time of the model run

# Set path to project directory
PATH_GRID   = '/data/SO2/SWOT/GRID/BIN/'                    # Space and time grid of the model 
PATH_OUTPUT = '/data/SO2/SWOT/MARA/RUN4_LY/DIAGS_HRLY/'     # Diagnostics of the model
PATH_nc     = '/data/SO3/lcolosi/mitgcm/SWOT_MARA_RUN4_LY/'  # Directory to save netCDFs 
PATH_figs   = '/home/lcolosi/AirSeaCoupling/figs_server/mitgcm/preliminary/'
file_dim    = '3D'                                          # Set the dimension of the data (to include the depth or not)

# Set plotting parameters 
fontsize = 14

Load the grid and diagnostics data into a python structure. The diagnostics that we will be looking at include: 

1. **Potential Temperature** $\theta$: $^\circ C$
2. **Salinity** $S$: $g/kg$
3. **Stratification** $\frac{d\sigma}{dz}$: $kg/m^4$
4. **Zonal, meridional, and vertical velocity components**  $\textbf{u} = (u,v,w)$: $m/s$  

In [4]:
# Create dataset 
ds = open_mdsdataset(
    PATH_OUTPUT,                    # File path where the model output data is stored (.data and .meta files)
    PATH_GRID,                      # File path to the grid data of the model 
    iters='all',                    # Specifies which iteration of the data to load
    delta_t=delta_t, 
    ignore_unknown_vars=False,      # Specifies whether to ignore any unknown variables that may appear in the dataset
    prefix=['diags_' + file_dim],   # List of prefixes to filter the variables in the dataset
    ref_date="2015-01-01 02:00:00", # Specifies the starting point of the simulation time (which may include the spin up time before diagnostics are output)
    geometry='sphericalpolar',      # Specifies the  grid's geometry is spherical-polar. 
    chunks={'i': 48, 'i_g': 48, 'j': 56,  'j_g': 56, 'k': 10, 'k_u':10, 'k_l':10, 'k_p1':10} # Chunck data upload into 48x56 pieces (spatial grid is divided into 12 chunks along the x and y axes) chunks={'i': 48, 'i_g': 48, 'j': 56,  'j_g': 56, 'k': 10, 'k_u':10, 'k_l':10, 'k_p1':10, 'time':1} chunks={'XC':48, 'YC':56, 'XG':48, 'YG':56, 'Z':10, 'Zp1':10, 'Zu':10, 'Zl':10, 'time':1}
)

# Convert all variables and coordinates in the dataset to little-endian (the format how multi-byte values are stored into memory)

#--- Variables ---#
for var in ds.data_vars:
    if ds[var].dtype.byteorder == '>' or (ds[var].dtype.byteorder == '=' and sys.byteorder == "big"):  # Check if big-endian
        ds[var] = ds[var].astype(ds[var].dtype.newbyteorder('<'))

#--- Coordinates ---# 
for coord in ds.coords:
    if ds[coord].dtype.byteorder == '>'or (ds[coord].dtype.byteorder == '=' and sys.byteorder == "big"):  # Check if big-endian
        ds[coord] = ds[coord].astype(ds[coord].dtype.newbyteorder('<'))

Slice the array based on latitude and longitude bounds and select the depth levels to analyze. 

In [5]:
ds_subset = ds.sel(
    XG=slice(*lon_bnds),
    YG=slice(*lat_bnds),
    XC=slice(*lon_bnds),
    YC=slice(*lat_bnds),
).isel(Z=0, Zl=0, Zp1=0, Zu=0)

Slice dataset for autocorrelation analysis in the following manner: 

1. Select a longitude and latitude point and get the time series at the point. For statistical convergence, grab time series of nearby longitude and latitude points (3x3 grid) 

2. Select spatial transects at one time step along a line of constant longitude and a line of constant latitude. For statistcal convergence, select the same transects for all time steps.

In [ ]:
# Set the slicing parameters (looking just on the offshelf for the present)
longitude_point = -122 % 360
latitude_point = 34
longitude = -122 % 360
latitude_b = [33, 34.25]
latitude = 34
longitude_b = [-123 % 360 , -121.75 % 360]

# Get the time series at a specific point and its adjacent points
ds_ts = ds_subset.sel(XG=slice(longitude_point-1, longitude_point+2), 
                      YG=slice(latitude_point-1, latitude_point+2),
                      XC=slice(longitude_point-1, longitude_point+2), 
                      YC=slice(latitude_point-1, latitude_point+2))

# Grab a transect at a constant longitude for all time steps
ds_nearest_lon = ds_subset.sel(XC=longitude, method='nearest')
ds_trans_zonal = ds_nearest_lon.sel(YC=slice(*latitude_b))

# Grab a transect at a constant latitude for all time steps
ds_nearest_lat = ds_subset.sel(YC=latitude, method='nearest')
ds_trans_meridional = ds_subset.sel(XC=slice(*longitude_b))

Save data 

In [ ]:
#--- Point Measurement Variables ---# 
# ds_ts['THETA'].to_netcdf(PATH_nc + 'THETA_CCS4_hrly_ts_ext.nc', encoding=encoding)
ds_ts['SALT'].to_netcdf(PATH_nc + 'SALT_CCS4_hrly_ts_ext.nc', encoding=encoding)

ds_ts['UVEL'].to_netcdf(PATH_nc + 'UVEL_CCS4_hrly_ts_ext.nc', encoding=encoding)
#ds_ts['VVEL'].to_netcdf(PATH_nc + 'VVEL_CCS4_hrly_ts_ext.nc', encoding=encoding)
ds_ts['WVEL'].to_netcdf(PATH_nc + 'WVEL_CCS4_hrly_ts_ext.nc', encoding=encoding)

# ds_ts['DRHODR'].to_netcdf(PATH_nc + 'Strat_CCS4_hrly_ts.nc', encoding=encoding)

# #--- Zonal Variables ---# 
# ds_trans_zonal['THETA'].to_netcdf(PATH_nc + 'THETA_CCS4_hrly_zonal.nc', encoding=encoding)
# ds_trans_zonal['SALT'].to_netcdf(PATH_nc + 'SALT_CCS4_hrly_zonal.nc', encoding=encoding)

# ds_trans_zonal['UVEL'].to_netcdf(PATH_nc + 'UVEL_CCS4_hrly_zonal.nc', encoding=encoding)
# ds_trans_zonal['VVEL'].to_netcdf(PATH_nc + 'VVEL_CCS4_hrly_zonal.nc', encoding=encoding)
# ds_trans_zonal['WVEL'].to_netcdf(PATH_nc + 'WVEL_CCS4_hrly_zonal.nc', encoding=encoding)

# ds_trans_zonal['DRHODR'].to_netcdf(PATH_nc + 'Strat_CCS4_hrly_zonal.nc', encoding=encoding)

# #--- Meridional Variables ---# 
# ds_trans_meridional['THETA'].to_netcdf(PATH_nc + 'THETA_CCS4_hrly_meridional.nc', encoding=encoding)
# ds_trans_meridional['SALT'].to_netcdf(PATH_nc + 'SALT_CCS4_hrly_meridional.nc', encoding=encoding)

# ds_trans_meridional['UVEL'].to_netcdf(PATH_nc + 'UVEL_CCS4_hrly_meridional.nc', encoding=encoding)
# ds_trans_meridional['VVEL'].to_netcdf(PATH_nc + 'VVEL_CCS4_hrly_meridional.nc', encoding=encoding)
# ds_trans_meridional['WVEL'].to_netcdf(PATH_nc + 'WVEL_CCS4_hrly_meridional.nc', encoding=encoding)

# ds_trans_meridional['DRHODR'].to_netcdf(PATH_nc + 'Strat_CCS4_hrly_meridional.nc', encoding=encoding)

/home/lcolosi/miniconda3/envs/ccs_scale/lib/python3.12/site-packages/xarray/backends/netCDF4_.py:585: UserWarning: endian-ness of dtype and endian kwarg do not match, using endian kwarg
  nc4_var = self.ds.createVariable(**default_args)
/home/lcolosi/miniconda3/envs/ccs_scale/lib/python3.12/site-packages/xarray/backends/netCDF4_.py:585: UserWarning: endian-ness of dtype and endian kwarg do not match, using endian kwarg
  nc4_var = self.ds.createVariable(**default_args)
